<a href="https://colab.research.google.com/github/hunterhuang18/My-Dumb-Financial-Analysis/blob/main/Portfolio_Returns_and_Risk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf

import warnings
warnings.filterwarnings('ignore')

In [6]:
### Download the data of equity in our portfolio

tickers = ['AAPL', 'META', 'NVDA', 'TSLA', 'MSFT']
portfolio = pd.DataFrame()
for ticker in tickers:
  df = yf.download(ticker, start = '2014-01-01', end = '2024-06-30')
  portfolio[ticker] = df['Adj Close']
portfolio.head()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,AAPL,META,NVDA,TSLA,MSFT
Date,,,,,
2014-01-02 00:00:00+00:00,17.234303,54.545769,0.373992,10.006667,31.120735
2014-01-03 00:00:00+00:00,16.855732,54.396221,0.369512,9.970667,30.911371
2014-01-06 00:00:00+00:00,16.947641,57.028294,0.374464,9.800000,30.258127
2014-01-07 00:00:00+00:00,16.826443,57.746132,0.380595,9.957333,30.492630
2014-01-08 00:00:00+00:00,16.933002,58.055199,0.385782,10.085333,29.948254


In [7]:
portfolio.describe()

,AAPL,META,NVDA,TSLA,MSFT
count,2640.000000,2640.000000,2640.000000,2640.000000,2640.000000
mean,80.684447,191.113593,13.652357,98.338908,156.562161
std,59.721426,100.274457,20.477679,107.735125,114.680767
min,15.572039,53.369308,0.362202,9.289333,29.295025
25%,27.776652,117.077499,1.482501,16.015334,51.455295
50%,48.969387,172.660149,5.546283,22.528000,113.090702
75%,141.549622,244.902634,16.688378,197.632500,249.472961
max,216.181656,526.314880,135.568405,409.970001,452.035248


In [8]:
portfolio.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2640 entries, 2014-01-02 00:00:00+00:00 to 2024-06-28 00:00:00+00:00
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2640 non-null   float64
 1   META    2640 non-null   float64
 2   NVDA    2640 non-null   float64
 3   TSLA    2640 non-null   float64
 4   MSFT    2640 non-null   float64
dtypes: float64(5)
memory usage: 123.8 KB


#1. Calculate the Returns of the Portfolio （通用)

In [9]:
### Calculate the reutnrs for each equity in the portfolio

portfolio_returns = portfolio.pct_change(1)
portfolio_returns.dropna(inplace = True)
portfolio_returns.head()

,AAPL,META,NVDA,TSLA,MSFT
Date,,,,,
2014-01-03 00:00:00+00:00,-0.021966,-0.002742,-0.011980,-0.003598,-0.006727
2014-01-06 00:00:00+00:00,0.005453,0.048387,0.013402,-0.017117,-0.021133
2014-01-07 00:00:00+00:00,-0.007151,0.012587,0.016373,0.016054,0.007750
2014-01-08 00:00:00+00:00,0.006333,0.005352,0.013631,0.012855,-0.017853
2014-01-09 00:00:00+00:00,-0.012770,-0.017345,-0.037286,-0.024788,-0.006431


In [10]:
### Create the weights for the stocks in the portfolio

num_stocks = 5
weights = [1/num_stocks] * num_stocks

In [11]:
### Calculate the covariance matrix and variances
portfolio_cov_matrix = portfolio_returns.cov()
portfolio_cov_matrix

portfolio_var = np.dot(np.transpose(weights), np.dot(portfolio_cov_matrix, weights))

### Standard deviation of the portfolio
portfolio_std = np.sqrt(portfolio_var)
print(f"The daily standard deviation of the portfolio is {portfolio_std}")

### Annual standard deviation
annual_portfolio_std = portfolio_std * np.sqrt(252)
print(f"The annual standard deviation of the portfolio is {annual_portfolio_std}")

### Standard deviation of each stock
stock_std = np.std(portfolio_returns) * np.sqrt(252)
stock_std

The daily standard deviation of the portfolio is 0.018634825959390366
The annual standard deviation of the portfolio is 0.2958186912811052


,0
AAPL,0.282085
META,0.376290
NVDA,0.466889
TSLA,0.555233
MSFT,0.267380


#2. Optimise weights to achieve a target return

In [12]:
from scipy.optimize import minimize

In [13]:
### Create the portfolio return function whose input is weights

def getPortReturn(weights):
  port_exp_ret = np.dot(np.transpose(weights), portfolio_returns.mean()) * 252
  return port_exp_ret

init_weights = [1/num_stocks] * num_stocks
getPortReturn(init_weights)

0.39074166111854897

In [14]:
### Find a combination of weights to acheive the target return 0.35

target_return = 0.35

bounds = tuple((0,1) for i  in range(num_stocks)) # Each weight is in range of 0 to 1
bounds

con = ({'type' : 'eq', 'fun': lambda w: np.sum(w)-1 }, # Constraint 1 : The sum of the weights should be equal to 1
      {'type' : 'eq', 'fun' : lambda x : x.dot(portfolio_returns.mean()) * 252 - target_return } # Constraint 2 : The return of the portfolio should be equal to the target return)

results = minimize(fun=getPortReturn, x0=init_weights, bounds=bounds, constraints=con)

In [15]:
results

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.3500000001108004
       x: [ 2.415e-01  2.395e-01  9.955e-02  1.822e-01  2.372e-01]
     nit: 2
     jac: [ 2.787e-01  2.839e-01  6.622e-01  4.388e-01  2.901e-01]
    nfev: 12
    njev: 2

In [16]:
optimized_weights = pd.DataFrame(results['x'])

In [17]:
optimized_weights.columns = ['Optimized Weights']
optimized_weights

,Optimized Weights
0,0.241453
1,0.239547
2,0.099547
3,0.182218
4,0.237235


#3. Minimising Portfolio Risk

In [ ]:
### Our objective is to minimize the objective function, which is the portfolio annual standard deviation, subject to two constraints: 1. weights should fall in the range of (0,1), 2. sum of weights should be equal to 1

### Define the function of generating the annualized standard deviation of our portfolio

def getPortRisk(weights):
  port_var = np.dot(np.transpose(weights), np.dot(portfolio_returns.cov(), weights))
  port_std = np.sqrt(port_var)
  port_std_annual = port_std * np.sqrt(252)
  return port_std_annual

# Set up the initial weights

num_stocks = len(tickers)
init_weights = [1/num_stocks] * num_stocks
getPortRisk(init_weights)

0.2958186594314112

In [ ]:
### Set up the bounds and constraints
bounds = tuple((0,1) for i in range(num_stocks))

cons = ({'type':'eq', 'fun': lambda x : np.sum(x) -1 })

In [ ]:
### Get the results through scipy, minimize

results = minimize(fun=getPortRisk, x0=init_weights, bounds=bounds, constraints=cons)
optimized_weights = pd.DataFrame(results['x'])
optimized_weights.columns = ['Optimized Weights']
optimized_weights

,Optimized Weights
0,0.393061
1,0.074820
2,0.000000
3,0.000261
4,0.531858


In [ ]:
getPortReturn(results['x'])

0.2852069278921947